<a href="https://colab.research.google.com/github/ashwinp15/astro_stats/blob/master/Bayesian_Evidence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/Spectral_Lag')

In [3]:
!pip install dynesty

     |████████████████████████████████| 92kB 3.0MB/s 


In [0]:
import pandas as pd
df = pd.read_csv("Data.csv")

In [5]:
df['X'] = df['X'].apply(lambda X:sum(map(int, X.split('-')))/2)
print(df.describe())

                  X          Y          E
count     37.000000  37.000000  37.000000
mean    3271.081081   1.755081   0.289676
std     4805.451130   1.205085   0.250918
min       14.000000  -0.223000   0.120000
25%      120.500000   0.913000   0.130000
50%      979.000000   1.856000   0.179000
75%     4256.500000   2.334000   0.290000
max    18494.000000   4.435000   1.030000


In [0]:
from numpy import exp
import scipy.integrate as integral
def model(E, theta, deg):
    f = lambda z: (1+z)**deg/((0.685+0.315*(1+z)**3)**0.5)
    i, unc = integral.quad(f, 0, 1.406)

    # for the case of no LIV
    if(deg==0):
        return theta[0]*(E**theta[1] - 11.34**theta[1])

    # for LIV of order n=1 and n=2
    return theta[0]*(E**theta[1] - 11.34**theta[1]) - (1+deg)*(E**deg - 11.34**deg)*i/(2*67.3*(theta[2]**deg))

In [0]:
import numpy as np
import dynesty

In [0]:
from scipy.stats import uniform
def prior(theta, deg):
  if deg==0:
    return np.array([uniform.ppf(theta[0], 0, 10),           # prior for tau      
                     uniform.ppf(theta[1], 0, 2)])
    
  return np.array([uniform.ppf(theta[0], 0, 10),           # prior for tau      
                   uniform.ppf(theta[1], 0, 2),           # prior for alpha
                   uniform.ppf(theta[2], 1e7, 1e16)])       # prior for E_QG

In [0]:
def loglike(theta, deg, df=df):
  E = df['X']
  t = df['Y']
  err = df['E']
  tM = model(E, theta, deg)
  return -0.5 * np.sum(np.log(2 * np.pi * err ** 2)
                         + (t - tM) ** 2 / err ** 2)


In [0]:
import functools
# this function takes the degree as input and returns the log likelihood function and prior for the corresponding model

def functional(flag, deg_i):
  if flag:                                                                    
    return functools.partial(loglike, deg=deg_i)
  return functools.partial(prior, deg=deg_i)

In [0]:
def calc_bayesian_evidence(degree):
  ndim = (degree!=0)+2
  loglike = functional(1, degree)
  prior = functional(0, degree)
  sampler = dynesty.NestedSampler(loglike, prior, ndim)
  return sampler


In [45]:
log_Bayes_Ev = []
for i in range(3):
  sampler = calc_bayesian_evidence(i)
  sampler.run_nested()
  result = sampler.results
  niter = result.niter
  log_Bayes_Ev.append(result.logz[niter-1])

4081it [00:52, 77.10it/s, +500 | bound: 9 | nc: 1 | ncall: 24591 | eff(%): 18.629 | loglstar:   -inf < -24.143 <    inf | logz: -31.429 +/-    nan | dlogz:  0.001 >  0.509]
4040it [01:12, 55.60it/s, +500 | bound: 8 | nc: 1 | ncall: 23492 | eff(%): 19.326 | loglstar:   -inf < -24.143 <    inf | logz: -31.347 +/-    nan | dlogz:  0.001 >  0.509]
3982it [01:09, 57.23it/s, +500 | bound: 7 | nc: 1 | ncall: 22793 | eff(%): 19.664 | loglstar:   -inf < -24.143 <    inf | logz: -31.233 +/-    nan | dlogz:  0.001 >  0.509]
